# Task 3 - Forecast Future Market Trends

### 1. Prepare for Forecasting
Retrain on full data for future predictions.

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model  # If loading saved model
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from datetime import timedelta

# Load full TSLA Adj Close
tsla_close = closing_prices['TSLA']  # From Task 1, or reload

# Scale full data
scaler = MinMaxScaler()
full_scaled = scaler.fit_transform(tsla_close.values.reshape(-1, 1))

# Reload or retrain LSTM (example retrain for simplicity)
lookback = 60
X_full, y_full = create_sequences(
    full_scaled, lookback)  # Use function from Task 2
X_full = X_full.reshape((X_full.shape[0], X_full.shape[1], 1))

model = Sequential()  # Rebuild and fit as in Task 2
# ... (add layers, compile, fit on X_full, y_full)

# Or load: model = load_model('lstm_model.h5')

### 2. Generate Forecast (6-12 Months)
Assume 252 trading days/year ~ 126 for 6 months, 252 for 12. We'll do 12 months for comprehensiveness.
For LSTM (iterative prediction: Use last lookback to predict next, append, repeat):

In [ ]:
# Forecast steps
forecast_steps = 252  # ~12 months trading days
future_dates = [tsla_close.index[-1] + timedelta(days=i) for i in range(1, forecast_steps + 1) if (
    tsla_close.index[-1] + timedelta(days=i)).weekday() < 5]  # Skip weekends
forecast_steps = len(future_dates)  # Adjust to actual trading days

# Start with last lookback
last_sequence = full_scaled[-lookback:].reshape(1, lookback, 1)
lstm_forecast = []

for _ in range(forecast_steps):
    pred = model.predict(last_sequence)[0][0]
    lstm_forecast.append(pred)
    last_sequence = np.append(last_sequence[:, 1:, :], [[[pred]]], axis=1)

# Inverse scale
lstm_forecast = scaler.inverse_transform(
    np.array(lstm_forecast).reshape(-1, 1)).flatten()

# Approximate confidence intervals (e.g., via simple std dev from residuals; for better, use bootstrapping)
# First, get residuals from training
train_preds_scaled = model.predict(X_full)
train_preds = scaler.inverse_transform(train_preds_scaled)
residuals = tsla_close.values[lookback:] - train_preds.flatten()
res_std = np.std(residuals)

# Rough CI: forecast ± 1.96 * res_std * sqrt(1 + cumsum(1/steps)) for widening
# Simplified: constant CI for illustration
ci_lower = lstm_forecast - 1.96 * res_std
ci_upper = lstm_forecast + 1.96 * res_std

#### For SARIMA (built-in intervals):

In [ ]:
# Forecast with CI
sarima_forecast_full = sarima_fit_full.get_forecast(steps=forecast_steps)
sarima_pred = sarima_forecast_full.predicted_mean
sarima_conf_int = sarima_forecast_full.conf_int()

# Future dates as above

### 3. Visualize the Forecast

In [ ]:
# Create forecast series
forecast_df = pd.DataFrame(
    {'Forecast': lstm_forecast, 'Lower CI': ci_lower, 'Upper CI': ci_upper}, index=future_dates)

# Plot historical + forecast
plt.figure(figsize=(14, 7))
# Last year for context
plt.plot(tsla_close[-365:], label='Historical (Last Year)')
plt.plot(forecast_df['Forecast'], label='Forecast')
plt.fill_between(forecast_df.index, forecast_df['Lower CI'],
                 forecast_df['Upper CI'], color='gray', alpha=0.3, label='95% CI')
plt.title('TSLA 12-Month Forecast with Confidence Intervals')
plt.xlabel('Date')
plt.ylabel('Adjusted Close Price')
plt.legend()
plt.show()